In [6]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

In [7]:
# Define the function to load the dataset
def load_path(path, part):
    dataset = []
    for folder in os.listdir(path):
        folder_path = os.path.join(path, folder)
        if os.path.isdir(folder_path):
            for body in os.listdir(folder_path):
                if body == part:
                    body_part = body
                    body_path = os.path.join(folder_path, body)
                    for patient_id in os.listdir(body_path):
                        patient_path = os.path.join(body_path, patient_id)
                        for lab in os.listdir(patient_path):
                            label = 'fractured' if lab.split('_')[-1] == 'positive' else 'normal'
                            lab_path = os.path.join(patient_path, lab)
                            for img in os.listdir(lab_path):
                                img_path = os.path.join(lab_path, img)
                                dataset.append({
                                    'body_part': body_part,
                                    'patient_id': patient_id,
                                    'label': label,
                                    'image_path': img_path
                                })
    return dataset


In [8]:
# Define the function to train the model for a specific body part
def trainPart(part):
    image_dir = './Dataset/'
    data = load_path(image_dir, part)
    labels = []
    filepaths = []

    for row in data:
        labels.append(row['label'])
        filepaths.append(row['image_path'])

    filepaths = pd.Series(filepaths, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    images = pd.concat([filepaths, labels], axis=1)

    train_df, test_df = train_test_split(images, train_size=0.9, shuffle=True, random_state=1)

    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        horizontal_flip=True,
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
        validation_split=0.2
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input
    )

    train_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='training'
    )

    val_images = train_generator.flow_from_dataframe(
        dataframe=train_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=64,
        shuffle=True,
        seed=42,
        subset='validation'
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df,
        x_col='Filepath',
        y_col='Label',
        target_size=(224, 224),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )

    pretrained_model = tf.keras.applications.ResNet50(
        input_shape=(224, 224, 3),
        include_top=False,
        weights='imagenet',
        pooling='avg'
    )

    pretrained_model.trainable = False

    inputs = pretrained_model.input
    x = tf.keras.layers.Dense(128, activation='relu')(pretrained_model.output)
    x = tf.keras.layers.Dense(50, activation='relu')(x)
    outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    print("-------Training " + part + "-------")

    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    history = model.fit(train_images, validation_data=val_images, epochs=25, callbacks=[callbacks])

    os.makedirs(f"./weights", exist_ok=True)
    model.save(f"./weights/ResNet50_{part}_frac.h5")
    results = model.evaluate(test_images, verbose=0)
    print(part + " Results:")
    print(results)
    print(f"Test Accuracy: {np.round(results[1] * 100, 2)}%")

    os.makedirs(f"./plots/FractureDetection/{part}", exist_ok=True)

    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f"./plots/FractureDetection/{part}/_Accuracy.jpeg")
    plt.clf()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.savefig(f"./plots/FractureDetection/{part}/_Loss.jpeg")
    plt.clf()

In [9]:
# Run the function for each body part
categories_parts = ["Elbow", "Hand", "Shoulder"]
for category in categories_parts:
    trainPart(category)

Found 3885 validated image filenames belonging to 2 classes.
Found 971 validated image filenames belonging to 2 classes.
Found 540 validated image filenames belonging to 2 classes.
-------Training Elbow-------


/home/arnav/anaconda3/envs/tf/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 28s 341ms/step - accuracy: 0.6264 - loss: 0.6563 - val_accuracy: 0.7405 - val_loss: 0.5411
Epoch 2/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 203ms/step - accuracy: 0.7669 - loss: 0.5095 - val_accuracy: 0.7652 - val_loss: 0.4997
Epoch 3/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 21s 211ms/step - accuracy: 0.7872 - loss: 0.4724 - val_accuracy: 0.7673 - val_loss: 0.5015
Epoch 4/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 14s 230ms/step - accuracy: 0.8160 - loss: 0.4299 - val_accuracy: 0.7724 - val_loss: 0.4945
Epoch 5/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 11s 182ms/step - accuracy: 0.8171 - loss: 0.4234 - val_accuracy: 0.7889 - val_loss: 0.4745
Epoch 6/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 211ms/step - accuracy: 0.8404 - loss: 0.3814 - val_accuracy: 0.7806 - val_loss: 0.4804
Epoch 7/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 14s 220ms/step - accuracy: 0.8361 - loss: 0.3893 - val_accuracy: 0.7951 - val_loss: 0.4708
Epoch 8/25
61/61 ━━━━━━━━━━━━━━━━━━━━ 13s 204ms/step - accuracy: 0.8584 - loss: 0.3565 - val_accu

Elbow Results:
[0.5015533566474915, 0.7777777910232544]
Test Accuracy: 77.78%
Found 4322 validated image filenames belonging to 2 classes.
Found 1080 validated image filenames belonging to 2 classes.
Found 601 validated image filenames belonging to 2 classes.
-------Training Hand-------


/home/arnav/anaconda3/envs/tf/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


...

/home/arnav/anaconda3/envs/tf/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 38s 318ms/step - accuracy: 0.6098 - loss: 0.6538 - val_accuracy: 0.7040 - val_loss: 0.5753
Epoch 2/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 27s 264ms/step - accuracy: 0.7140 - loss: 0.5689 - val_accuracy: 0.7251 - val_loss: 0.5528
Epoch 3/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 24s 240ms/step - accuracy: 0.7358 - loss: 0.5324 - val_accuracy: 0.7264 - val_loss: 0.5446
Epoch 4/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 26s 251ms/step - accuracy: 0.7629 - loss: 0.5034 - val_accuracy: 0.7481 - val_loss: 0.5204
Epoch 5/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 24s 241ms/step - accuracy: 0.7661 - loss: 0.4973 - val_accuracy: 0.7512 - val_loss: 0.5139
Epoch 6/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 23s 226ms/step - accuracy: 0.7834 - loss: 0.4721 - val_accuracy: 0.7525 - val_loss: 0.5163
Epoch 7/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 25s 248ms/step - accuracy: 0.7905 - loss: 0.4592 - val_accuracy: 0.7469 - val_loss: 0.5157
Epoch 8/25
101/101 ━━━━━━━━━━━━━━━━━━━━ 25s 247ms/step - accuracy: 0.7930 - loss: 0

2024-11-12 14:49:00.405762: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1712', 28 bytes spill stores, 28 bytes spill loads

2024-11-12 14:49:00.561392: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1712', 176 bytes spill stores, 176 bytes spill loads



Shoulder Results:
[0.49960067868232727, 0.7751677632331848]
Test Accuracy: 77.52%


<Figure size 640x480 with 0 Axes>